In [1]:
!apt-get install tesseract-ocr -y
!pip install pytesseract opencv-python pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import cv2
import pytesseract
import re
from collections import defaultdict
from sklearn.cluster import DBSCAN
import numpy as np

# -------------------------------------------------------------------
# OCR HELPER 1 — LINES (for invoice number & date)
# -------------------------------------------------------------------

def ocr_image_to_lines(image_path: str):
    img = cv2.imread(image_path)
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    line_map = defaultdict(list)

    for i in range(len(data["text"])):
        txt = data["text"][i].strip()
        conf = int(data["conf"][i])
        if conf < 0 or txt == "":
            continue

        key = (data["block_num"][i], data["par_num"][i], data["line_num"][i])
        line_map[key].append((data["left"][i], data["top"][i], txt))

    lines = []
    for key, arr in line_map.items():
        arr = sorted(arr, key=lambda x: x[0])
        text = " ".join([w[2] for w in arr])
        y = int(sum([w[1] for w in arr]) / len(arr))
        if text:
            lines.append({"text": text, "y": y})

    lines = sorted(lines, key=lambda x: x["y"])
    return lines


# -------------------------------------------------------------------
# OCR HELPER 2 — WORDS (for line items)
# -------------------------------------------------------------------

def ocr_words(image_path):
    img = cv2.imread(image_path)
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    words = []
    for i in range(len(data["text"])):
        txt = data["text"][i].strip()
        conf = int(data["conf"][i])
        if conf < 0 or txt == "":
            continue

        x, y, w, h = (
            data["left"][i], data["top"][i],
            data["width"][i], data["height"][i]
        )

        words.append({
            "text": txt,
            "x1": x,
            "y1": y,
            "x2": x + w,
            "y2": y + h,
            "cy": y + h/2
        })

    return words


# -------------------------------------------------------------------
# NUMBER PARSING HELPERS
# -------------------------------------------------------------------

def parse_decimal(s: str) -> float:
    s = s.replace('.', '')
    s = s.replace(',', '.')
    return float(s)

def format_decimal(v: float) -> str:
    return f"{v:.2f}".replace('.', ',')


# -------------------------------------------------------------------
# BUILD RAW ITEM LINES FROM WORDS
# -------------------------------------------------------------------

def build_raw_item_lines(words):
    """
    Robust multi-row extraction using DBSCAN clustering on Y (cy).
    Fixes:
    - missing row numbers (3. missing in your OCR)
    - multi-line descriptions
    - uneven spacing
    - row mixing
    """

    # 1) Locate header and summary
    header_y = None
    summary_y = None

    for w in words:
        t = w["text"].lower()
        if t in ["no", "no.", "description", "qty", "qty."]:
            header_y = w["y1"]
        if "summary" in t:
            summary_y = w["y1"]

    if header_y is None:
        return []

    if summary_y is None:
        summary_y = 999999

    # 2) Collect words inside table region
    table_words = [w for w in words if header_y < w["cy"] < summary_y]
    if not table_words:
        return []

    # 3) Cluster words by vertical axis using DBSCAN
    cy_values = np.array([[w["cy"]] for w in table_words])
    clustering = DBSCAN(eps=40, min_samples=1).fit(cy_values)

    # 4) Put cluster words together
    clusters = {}
    for label, w in zip(clustering.labels_, table_words):
        clusters.setdefault(label, []).append(w)

    # 5) Sort clusters from top to bottom
    sorted_clusters = sorted(clusters.values(), key=lambda grp: np.mean([w["cy"] for w in grp]))

    # 6) Build rows
    merged_rows = []
    current_number = 1

    for grp in sorted_clusters:
        grp_sorted = sorted(grp, key=lambda w: w["x1"])
        text = " ".join(w["text"] for w in grp_sorted)

        # detect explicit row number if present
        number_match = re.match(r"\d+[\.\):]?", grp_sorted[0]["text"])
        if number_match:
            current_number = int(re.findall(r"\d+", grp_sorted[0]["text"])[0])
        else:
            # assign missing row numbers (fixes your missing 3.)
            text = f"{current_number}. " + text

        merged_rows.append(text)
        current_number += 1

    return merged_rows






# -------------------------------------------------------------------
# PARSE ONE ITEM LINE
# -------------------------------------------------------------------

def parse_item_line(line: str):
    m_no = re.match(r"^\s*(\d+)\s*[\.\)]?\s+", line)
    if not m_no:
        return None

    no = int(m_no.group(1))
    rest = line[m_no.end():].strip()

    num_pattern = r"\d{1,3}(?:[.,]\d{3})*[.,]\d{2}"
    nums = list(re.finditer(num_pattern, rest))
    if len(nums) < 2:
        return None

    qty_str = nums[0].group(0)
    price_str = nums[1].group(0)
    description = rest[:nums[0].start()].strip()

    m_vat = re.search(r"(\d+)\s*%", rest)
    if m_vat:
        vat_percent = m_vat.group(1) + "%"
        vat_val = float(m_vat.group(1))
    else:
        vat_percent = "0%"
        vat_val = 0.0

    qty = parse_decimal(qty_str)
    price = parse_decimal(price_str)

    net = qty * price
    gross = net * (1 + vat_val / 100)

    return {
        "no": no,
        "description": description,
        "qty": qty_str,
        "net_price": price_str,
        "net_worth": format_decimal(net),
        "vat_percent": vat_percent,
        "gross_worth": format_decimal(gross)
    }


# -------------------------------------------------------------------
# LINE ITEM PIPELINE
# -------------------------------------------------------------------

def extract_line_items(image_path):
    words = ocr_words(image_path)
    raw_rows = build_raw_item_lines(words)

    items = []
    for row in raw_rows:
        parsed = parse_item_line(row)
        if parsed:
            items.append(parsed)

    return items


# -------------------------------------------------------------------
# INVOICE NUMBER & DATE (old working logic)
# -------------------------------------------------------------------

def extract_invoice_number(lines):
    for line in lines:
        txt = line["text"]
        if "invoice" in txt.lower():
            m = re.search(r"(\d{4,20})", txt)
            if m:
                return m.group(1)
    return ""

def extract_invoice_date(lines):
    pattern = r"\d{1,2}[\/\-.]\d{1,2}[\/\-.]\d{2,4}"

    for line in lines:
        if "date" in line["text"].lower():
            m = re.search(pattern, line["text"])
            if m:
                return m.group(0)

    for line in lines:
        m = re.search(pattern, line["text"])
        if m:
            return m.group(0)

    return ""


# -------------------------------------------------------------------
# MAIN WRAPPER
# -------------------------------------------------------------------

def extract_invoice(image_path):
    lines = ocr_image_to_lines(image_path)      # good for invoice no/date
    items = extract_line_items(image_path)      # new word-based extractor

    return {
        "invoice_number": extract_invoice_number(lines),
        "invoice_date": extract_invoice_date(lines),
        "line_items": items
    }


# -------------------------------------------------------------------
# TEST
# -------------------------------------------------------------------

# if __name__ == "__main__":
#     img = "/content/batch1-0031.jpg"  # change path
#     from pprint import pprint
#     pprint(extract_invoice(img))


In [3]:
import os
import json

# -------------------------------------------------------------
# Function from previous step: Saves your desired JSON structure
# -------------------------------------------------------------
def build_invoice_json(extraction_result, output_path):
    final_json = {
        "invoice_number": extraction_result.get("invoice_number", ""),
        "invoice_date": extraction_result.get("invoice_date", ""),
        "line_items": []
    }

    for item in extraction_result.get("line_items", []):
        final_json["line_items"].append({
            "no": item.get("no", ""),
            "description": item.get("description", ""),
            "qty": item.get("qty", ""),
            "net_price": item.get("net_price", ""),
            "net_worth": item.get("net_worth", ""),
            "vat_percent": item.get("vat_percent", ""),
            "gross_worth": item.get("gross_worth", "")
        })

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(final_json, f, indent=4, ensure_ascii=False)

    print(f"✔ JSON saved → {output_path}")


# -------------------------------------------------------------
#  NEW FUNCTION → Process ALL images in folder automatically
# -------------------------------------------------------------
def process_all_invoices(image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    supported_ext = [".jpg", ".jpeg", ".png", ".tiff"]

    images = [
        f for f in os.listdir(image_folder)
        if os.path.splitext(f)[1].lower() in supported_ext
    ]

    images.sort()  # ensures stable order: invoice_0001, invoice_0002 …

    print(f"Found {len(images)} invoice images.")

    for idx, img_name in enumerate(images, start=1):
        img_path = os.path.join(image_folder, img_name)

        print(f"\n🔍 Processing: {img_name}")

        # ---- RUN YOUR EXTRACTION MODEL ----
        extraction_result = extract_invoice(img_path)

        # Output JSON name → invoice_0001.json etc.
        json_name = f"batch1-{idx:04d}.json"
        json_path = os.path.join(output_folder, json_name)

        # ---- SAVE JSON ----
        build_invoice_json(extraction_result, json_path)

    print("\n🎉 ALL INVOICES PROCESSED SUCCESSFULLY!")


In [19]:
image_folder = "/content/drive/MyDrive/AssignmentDataset"          # folder containing all invoice images
output_folder = "/content/drive/MyDrive/AssignmentDataset"    # folder where JSON files should be saved

process_all_invoices(image_folder, output_folder)


Found 391 invoice images.

🔍 Processing: batch1-0001.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0001.json

🔍 Processing: batch1-0002.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0002.json

🔍 Processing: batch1-0003.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0003.json

🔍 Processing: batch1-0004.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0004.json

🔍 Processing: batch1-0005.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0005.json

🔍 Processing: batch1-0006.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0006.json

🔍 Processing: batch1-0007.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0007.json

🔍 Processing: batch1-0008.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0008.json

🔍 Processing: batch1-0009.jpg
✔ JSON saved → /content/drive/MyDrive/AssignmentDataset/batch1-0009.json

🔍 Processing: batch1-0010.jpg
✔ JSON 

In [4]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
from PIL import Image


In [5]:
CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,:/-_| "
PAD = "<PAD>"
SOS = "<SOS>"
EOS = "<EOS>"

VOCAB = [PAD, SOS, EOS] + list(CHARS)
char2idx = {c:i for i,c in enumerate(VOCAB)}
idx2char = {i:c for c,i in char2idx.items()}


In [6]:
MAX_INV_NUM = 20
MAX_INV_DATE = 20
MAX_LINE_ITEMS = 300

In [7]:
class InvoiceEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        m = resnet18(pretrained=True)
        m.fc = nn.Identity()
        self.cnn = m
        self.out_features = 512

    def forward(self, x):
        return self.cnn(x)


In [8]:
class SeqHead(nn.Module):
    def __init__(self, encoder_dim, vocab_size, max_len):
        super().__init__()
        self.max_len = max_len

        self.fc = nn.Linear(encoder_dim, 512)
        self.lstm = nn.LSTM(512, 512, batch_first=True)
        self.classifier = nn.Linear(512, vocab_size)

    def forward(self, features, targets=None):
        B = features.size(0)

        x = self.fc(features)                 # (B,512)
        x = x.unsqueeze(1).repeat(1, self.max_len, 1)

        x, _ = self.lstm(x)                   # (B,max_len,512)

        logits = self.classifier(x)           # (B,max_len,vocab)

        return logits


In [9]:
class InvoiceModel(nn.Module):
    def __init__(self, train_inv_num=True, train_inv_date=True, train_line_items=True):
        super().__init__()
        self.encoder = InvoiceEncoder()

        self.train_inv_num = train_inv_num
        self.train_inv_date = train_inv_date
        self.train_line_items = train_line_items

        if train_inv_num:
            self.inv_num_head = SeqHead(512, len(VOCAB), MAX_INV_NUM)

        if train_inv_date:
            self.inv_date_head = SeqHead(512, len(VOCAB), MAX_INV_DATE)

        if train_line_items:
            self.line_items_head = SeqHead(512, len(VOCAB), MAX_LINE_ITEMS)

    def forward(self, images):
        feat = self.encoder(images)

        outputs = {}
        if self.train_inv_num:
            outputs["inv_number"] = self.inv_num_head(feat)
        if self.train_inv_date:
            outputs["inv_date"] = self.inv_date_head(feat)
        if self.train_line_items:
            outputs["line_items"] = self.line_items_head(feat)

        return outputs


In [10]:
def encode_seq(text, max_len):
    seq = [char2idx[SOS]]
    for ch in text:
        if ch in char2idx:
            seq.append(char2idx[ch])
    seq.append(char2idx[EOS])

    # pad
    seq += [char2idx[PAD]] * (max_len - len(seq))
    return seq[:max_len]

class InvoiceDataset(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.files = [f[:-5] for f in os.listdir(folder) if f.endswith(".json")]

        self.transform = T.Compose([
            T.Resize((256,256)),
            T.ToTensor(),
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        base = self.files[idx]

        img_path = os.path.join(self.folder, base + ".jpg")
        json_path = os.path.join(self.folder, base + ".json")

        image = Image.open(img_path).convert("RGB")
        with open(json_path) as f:
            data = json.load(f)

        inv_num = data.get("invoice_number", "")
        inv_date = data.get("invoice_date", "")
        line_items = data.get("line_items", [])

        # Serialize line items
        line_str = ""
        for item in line_items:
            part = f"{item['no']}|{item['description']}|{item['qty']}|{item['net_price']};"
            line_str += part
        line_str = line_str[:MAX_LINE_ITEMS]

        return {
            "image": self.transform(image),
            "inv_number_seq": torch.tensor(encode_seq(inv_num, MAX_INV_NUM)),
            "inv_date_seq": torch.tensor(encode_seq(inv_date, MAX_INV_DATE)),
            "line_items_seq": torch.tensor(encode_seq(line_str, MAX_LINE_ITEMS)),
        }


In [11]:
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    loss_fn = nn.CrossEntropyLoss(ignore_index=char2idx[PAD])
    total_loss = 0

    for batch in loader:
        imgs = batch["image"].to(device)
        out = model(imgs)

        loss = 0
        if model.train_inv_num:
            t = batch["inv_number_seq"].to(device)
            l = out["inv_number"]
            loss += loss_fn(l.transpose(1,2), t)

        if model.train_inv_date:
            t = batch["inv_date_seq"].to(device)
            l = out["inv_date"]
            loss += loss_fn(l.transpose(1,2), t)

        if model.train_line_items:
            t = batch["line_items_seq"].to(device)
            l = out["line_items"]
            loss += loss_fn(l.transpose(1,2), t)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [12]:
dataset = InvoiceDataset("/content/drive/MyDrive/AssignmentDataset")
loader  = DataLoader(dataset, batch_size=4, shuffle=True)

model = InvoiceModel(
    train_inv_num=True,
    train_inv_date=True,
    train_line_items=True
).to("cuda")

optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(20):
    loss = train_one_epoch(model, loader, optimizer, "cuda")
    print(f"Epoch {epoch+1}: loss={loss:.4f}")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]


Epoch 1: loss=8.6797
Epoch 2: loss=7.4907
Epoch 3: loss=7.2042
Epoch 4: loss=7.0199
Epoch 5: loss=6.8241
Epoch 6: loss=6.5483
Epoch 7: loss=6.3739
Epoch 8: loss=6.2630
Epoch 9: loss=6.1897
Epoch 10: loss=6.1375
Epoch 11: loss=6.0964
Epoch 12: loss=6.0583
Epoch 13: loss=6.0099
Epoch 14: loss=5.9992
Epoch 15: loss=5.9537
Epoch 16: loss=5.9240
Epoch 17: loss=5.9071
Epoch 18: loss=5.8816
Epoch 19: loss=5.8453
Epoch 20: loss=5.8456


In [13]:
def decode_seq(seq):
    chars = []
    for idx in seq:
        ch = idx2char[int(idx)]
        if ch == EOS: break
        if ch not in [PAD, SOS]:
            chars.append(ch)
    return "".join(chars)

def predict(model, img_path):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    img = T.Resize((256,256))(img)
    img = T.ToTensor()(img).unsqueeze(0).to("cuda")

    with torch.no_grad():
        out = model(img)

    result = {}

    if "inv_number" in out:
        seq = out["inv_number"].argmax(-1)[0]
        result["invoice_number"] = decode_seq(seq)

    if "inv_date" in out:
        seq = out["inv_date"].argmax(-1)[0]
        result["invoice_date"] = decode_seq(seq)

    if "line_items" in out:
        seq = out["line_items"].argmax(-1)[0]
        line_str = decode_seq(seq)

        # Convert back to JSON
        items = []
        for part in line_str.split(";"):
            if "|" in part:
                try:
                    no, desc, qty, price = part.split("|")
                    items.append({
                        "no": no, "description": desc,
                        "qty": qty, "net_price": price
                    })
                except:
                    pass
        result["line_items"] = items

    return result


In [15]:
result = predict(model, "/content/batch1-1002.jpg")
print(result)

{'invoice_number': '68888333', 'invoice_date': '01/21/2012', 'line_items': []}
